# Estructurando un Proyecto de DS (Parte II)

### Abstract

El conjunto de datos que se analiza en este estudio se centra en la reserva de hoteles y la probabilidad de que estas reservas sean canceladas. El dataset contiene información detallada sobre las reservas realizadas en distintos hoteles, incluyendo información sobre el tipo de habitación reservada, la duración de la estancia, la anticipación de la reserva, la fecha de llegada, la cantidad de personas y dias que reservan, entre otros datos.

- ¿Afecta el tipo de mercado el estado final de la reserva?

- ¿Existe relación entre el estado de la reserva y el tiempo de llegada al hotel?

- ¿Hay alguna época en particular en la que se realizan mayor cantidad de cancelaciones?

### Contexto y Problemática Comercial

#### Contexto Comercial y Analítico

El sector turístico es uno de los más importantes y dinámicos de la economía global, y los hoteles son una parte fundamental de esta industria. Los hoteles compiten en un mercado altamente competitivo, por lo que lograr realizar predicciones sobre la cancelación de las reservas realizadas pueden ayudarlos a mantenerse competitivos.

Los diferentes métodos de reserva que existen (y principalmente la reserva online) afectaron fuertemente el comportamiento de los clientes y sus posibilidades de reservas, muchas de las cuales luego son canceladas o no asistidas. Generalmente, el ofrecimiento de cancelaciones gratuitas o a un precio moderado/bajo ayuda a que los huespedes se vean beneficiados, pero conlleva a que los hoteles pierdan rentabilidad ya que para ellos esta metodología de cancelaciones es costosa.

#### Problemática


La cancelación de reservas es un problema común en la industria hotelera y puede afectar significativamente la rentabilidad del hotel. Estas cancelaciones pueden generar costos adicionales para el hotel, como la pérdida de ingresos por habitaciones no vendidas, gastos de cancelación y posibles daños a la reputación del hotel si el cliente queda insatisfecho.

Por lo tanto, es crucial para los hoteles gestionar eficazmente las cancelaciones de reservas para minimizar su impacto en la rentabilidad y la satisfacción del cliente.

Mediante este dataset se busca generar un modelo que ayude a la industria hotelera a gestionar las cancelaciones de las reservas. El dataset tiene mas de 36000 reservas analizadas y diferenciadas en canceladas y no canceladas, ademas de caracteristicas adicionales de las personas, la estadia y las reservas.

### Objetivo

El objetivo principal de este estudio es analizar la probabilidad de que las reservas de hotel sean canceladas y determinar qué factores pueden influir en esta decisión. Para lograr este objetivo, se llevaran a cabo varios análisis estadísticos y van a utilizarse diferentes técnicas de modelado predictivo que nos permitira generar un modelo de clasificación para predecir las posibilidades de que esto ocurra.

### Importación de Librerias

In [ ]:
#Numpy
import numpy as np
from numpy import median
# Pandas
import pandas as pd
# Matplotlib
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns
# Sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics

### Dataset

#### Importación y Lectura

#### Visualización

In [ ]:
pd.set_option('display.max_columns', 500)

Link al dataset: https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset

In [ ]:
hotel_orig = pd.read_csv('./ReservaHotel.csv')

In [ ]:
hotel_orig.head()

In [ ]:
hotel_orig.shape

### Data Wrangling

In [ ]:
hotel_orig.info()

Las variables del dataset son:
- Booking_ID: ID de reserva.
- no_of_adults: número de adultos.
- no_of_children: número de niños.
- no_of_weekend_nights: número de noches de fin de semana reservado.
- no_of_week_nights: número de noches de dias de semana reservado.
- type_of_meal_plan: plan de alimentación elegido.
- required_car_parking_space: si requiere estacionamiento o no.
- room_type_reserved: tipo de habitación reservada.
- lead_time: cantidad de dias en que hace la reservación.
- arrival_year: año de llegada al hotel.
- arrival_month: mes de llegada al hotel.
- arrival_date: dia de llegada al hotel.
- market_segment_type: lugar desde el que se realizo la reserva.
- repeated_guest: si la persona que reserva ya habia reservado con anterioridad.
- no_of_previous_cancellations: cantidad de cancelaciones previas.
- no_of_previous_bookings_not_canceled: cantidad de reservas previas no canceladas.
- avg_price_per_room: precio promedio por habitación.
- no_of_special_requests: cantidad de pedidos especiales.
- booking_status: estado final de la reserva.

#### Datos faltantes

In [ ]:
hotel_orig.isna().sum()

El dataset no posee datos faltantes.

#### Primera Limpieza de Variables

Inicialmente la unica variable a eliminar sera el ID de reserva ya que es un dato irrelevante para el análisis.
Se generara un dataframe nuevo para mantener el original intacto.

In [ ]:
hotel = hotel_orig.drop(columns=['Booking_ID'], axis=1)

In [ ]:
hotel.head()

In [ ]:
hotel.describe(include="all")

#### Creación de Variable datetime ("Date")

In [ ]:
hotel['Date'] = pd.to_datetime({'year':hotel['arrival_year'],
                              'month':hotel['arrival_month'],    
                              'day':hotel['arrival_date']
                              })

In [ ]:
hotel.head()

In [ ]:
hotel.info()

Para conocer el período de reservas que abarca el dataset realizamos lo siguiente:

In [ ]:
max_fecha = hotel['Date'].max()
min_fecha = hotel['Date'].min()
print('Las reservas analizadas en el dataset van desde {} a {}'.format(min_fecha, max_fecha))

#### Nuevas Variables

Se crearan nuevas variables a partir de la unificación de varias columnas.

In [ ]:
# unifico las variables de adults y children para ver el total de personas.
hotel['total_people'] = hotel['no_of_adults'] + hotel['no_of_children']

# unifico el numero de nights para ver su total
hotel['total_nights'] = hotel['no_of_weekend_nights'] + hotel['no_of_week_nights']

# unifico el numero de reservas anteriores
hotel['previous_booking'] = hotel['no_of_previous_bookings_not_canceled'] + hotel['no_of_previous_cancellations']

In [ ]:
hotel.head()

In [ ]:
hotel.shape

#### Transformación de variables categóricas a numéricas

In [ ]:
hotel.head()

In [ ]:
hotel.type_of_meal_plan.unique()

In [ ]:
map_meal = {'Not Selected': 0, 'Meal Plan 1': 1, 'Meal Plan 2': 2, 'Meal Plan 3': 3}
hotel["type_of_meal_plan"] = hotel["type_of_meal_plan"].replace(map_meal)
hotel["type_of_meal_plan"]  

In [ ]:
hotel.room_type_reserved.unique()

In [ ]:
map_room = {'Room_Type 1': 1, 'Room_Type 4': 4, 'Room_Type 2': 2, 'Room_Type 6': 6,
       'Room_Type 5': 5, 'Room_Type 7': 7, 'Room_Type 3': 3}
hotel["room_type_reserved"] = hotel["room_type_reserved"].replace(map_room)
hotel["room_type_reserved"]

In [ ]:
hotel.booking_status.unique()

In [ ]:
map_booking = {'Not_Canceled': 0, 'Canceled': 1}
hotel["booking_status"] = hotel["booking_status"].replace(map_booking)
hotel["booking_status"]

In [ ]:
hotel_segment = pd.get_dummies(hotel["market_segment_type"], prefix="market")
hotel_segment.head()

In [ ]:
hotel = hotel.drop(["market_segment_type"], axis = 1)
hotel.head()

In [ ]:
hotel_concat = pd.concat([hotel, hotel_segment], axis = 1)
hotel = hotel_concat
hotel.head()

In [ ]:
hotel.shape

In [ ]:
hotel.info()

#### Cambio de tipos de variables

In [ ]:
hotel.booking_status = hotel.booking_status.astype('object')

In [ ]:
hotel.market_Aviation = hotel.market_Aviation.astype('object')
hotel.market_Complementary = hotel.market_Complementary.astype('object')
hotel.market_Corporate = hotel.market_Corporate.astype('object')
hotel.market_Offline = hotel.market_Offline.astype('object')
hotel.market_Online = hotel.market_Online.astype('object')

In [ ]:
hotel.repeated_guest = hotel.repeated_guest.astype('object')
hotel.type_of_meal_plan = hotel.type_of_meal_plan.astype('object')
hotel.room_type_reserved = hotel.room_type_reserved.astype('object')
hotel.required_car_parking_space = hotel.required_car_parking_space.astype('object')

In [ ]:
hotel.info()

#### Gráficos descriptivos

In [ ]:
hs = sns.countplot(data=hotel, x= "booking_status", palette='deep')
hs.set_title('Cantidad de reservas según su estado')
labels = ('No cancelada', 'Cancelada')
hs.set_ylabel('Cantidad de resesrvas')
hs.set_xlabel('Estado de reserva')
hs.set_xticklabels(labels)
plt.show()

In [ ]:
hotel['booking_status'].value_counts().mul(100)/len(hotel)

*De acuerdo a lo observado, las reservas NO CANCELADAS alcanzan un total aproximado de 67%, mientras que las reservas CANCELADAS son de alrededor del 33%.*

In [ ]:
hl = sns.boxplot(x='booking_status', y='lead_time', data = hotel)
sns.set(font_scale=1)
hl.set_ylabel('Tiempo de espera')
hl.set_xlabel('Estado de reserva')
hl.set_xticklabels(labels)
plt.title("Lead Time vs. Status")

In [ ]:
hotel['lead_time'].describe()

*En función de lo que visualizamos, podemos determinar que a mayor cantidad de dias de espera para el arribo al hotel, mayor es la tasa de cancelaciones, llegando incluso a visualizar "outliers" con valores mayores a 400 dias. Sin embargo, hay reservas con tiempos de espera menores a esa cantidad que no fueron canceladas*

#### Revisión de tipos de variables

In [ ]:
hotel.dtypes

Armare 2 dataframes de acuerdo al tipo de variable a analizar, uno con las variables cualitativas y otra con las variables cuantitativas.

#### Variables Cuantitativas

In [ ]:
hotel_cuant = hotel.select_dtypes(include = np.number)
hotel_cuant.info()

In [ ]:
hotel_cuant.describe()

In [ ]:
for i in hotel_cuant:
    plt.figure(figsize=(18, 6))
    if hotel_cuant[i].nunique() <= 10:
        sns.countplot(x=i, data=hotel_cuant, hue=hotel["booking_status"])
    else:
        sns.histplot(x=i, data=hotel_cuant, hue=hotel["booking_status"])
    
    sns.set(font_scale=1)
    plt.show()


Tomaremos la variable "Arrival_month" para hacer un análisis específico de la misma.

In [ ]:
hotel = hotel.sort_values(["arrival_month"])
hotel_cuant['arrival_month'].unique()

In [ ]:
map_month = {10: "Octubre", 11: "Noviembre",  2: "Febrero",  5: "Mayo",  4: "Abril",  9: "Septiembre", 12: "Diciembre",  7: "Julio",  6: "Junio",  8: "Agosto",  3: "Marzo",  1: "Enero"}
hotel["arrival_month"] = hotel["arrival_month"].replace(map_month)
hotel["arrival_month"].unique()

In [ ]:
plt.figure(figsize=(15,8))
sns.set_context("poster",font_scale = .56)
sns.countplot(x = hotel['arrival_month'],data = hotel , hue='booking_status')
plt.show()

* A partir del mes de Agosto y hasta el mes de Diciembre las reservas crecen significativamente, coincidentemente con las vacaciones de invierno.
* Durante los meses de Diciembre, Enero y Febrero las reservas canceladas son minimas.

#### Variables Cualitativas

In [ ]:
hotel_cuali = hotel.select_dtypes(include= object)
hotel_cuali.info()

In [ ]:
for col in hotel_cuali.columns:
    print(hotel_cuali[col].unique())
    print(hotel_cuali[col].nunique())

In [ ]:
for i in hotel_cuali:
    plt.figure(figsize=(18, 6))
    sns.countplot(x =i, data = hotel_cuali, palette = 'bright', hue = hotel["booking_status"])
    sns.set(font_scale = 1)
    plt.show()  


In [ ]:
ht_meal = pd.crosstab(hotel_cuali.booking_status, hotel_cuali.type_of_meal_plan, normalize='columns')
ht_meal

In [ ]:
ht_meal.plot(kind='bar') 
plt.grid(color = 'w')
plt.xlabel ('Estado de la reserva')
plt.ylabel ('Frecuencia relativa')
plt.show()

In [ ]:
ht_room = pd.crosstab(hotel_cuali.booking_status, hotel_cuali.room_type_reserved, normalize='columns')
ht_room

In [ ]:
ht_room.plot(kind='bar') 
plt.grid(color = 'w')
plt.xlabel ('Estado de la reserva')
plt.ylabel ('Frecuencia relativa')
plt.show()

In [ ]:
htcancel = hotel["booking_status"]
htcancel

In [ ]:
hotel_cuant["iscancel"] = htcancel
hotel_cuant.info() 

In [ ]:
#sns.pairplot(hotel_cuant, hue='iscancel')

#### Conclusiones

- Las personas que realizaron reservas previas en el mismo hotel no registran cancelaciones.
- Se observan 2 nubes bien diferenciadas para el precio promedio de habitación (avg_price_per_room) y para el tiempo de espera (lead_time).
- Aproximadamente el 50% de las reservas que comprenden el plan 2 de comidas (type_of_meal_plan) son canceladas.
- Las reservas realizadas se mantienen para 9 de cada 10 en la categoria "Corporate" (market_segment_type)
- Se observa que la variable target sería "booking_status" para ver la cancelación de las reservas.


### Exploratory Data Analysis

#### Análisis Univariado

##### Variables Cuantitativas

In [ ]:
hotel_cuant.info()

In [ ]:
hotel_cuant = hotel_cuant.drop(['iscancel','arrival_year','arrival_month','arrival_date'], axis=1)

In [ ]:
for i in hotel_cuant:
    hotel_cuant.boxplot(column = i)
    plt.show()

Gran parte de los gráficos muestran las cajas colapsadas, observandose de esta manera muchos outliers los cuales no nos permiten obtener información relevante. De igual manera analizaremos algunas de estas variables en detalle mas abajo.

##### Variables Cualitativas

In [ ]:
hotel_cuali.info()

In [ ]:
hotel_cuali = hotel_cuali.drop(['arrival_month'], axis = 1)
hotel_cuali.info()

In [ ]:
for col in hotel_cuali.columns:
    print(hotel_cuali[col].unique())
    print(hotel_cuali[col].nunique())  

In [ ]:
for i in hotel_cuali:
    plt.figure(figsize=(18, 6))
    sns.countplot(x =i, data = hotel, palette = 'deep')
    sns.set(font_scale = 1)
    plt.show()  

Analizaremos con profundidad algunas variables vistas previamente.

In [ ]:
hotel.info()

##### Variable target "booking_status"

In [ ]:
hotel['booking_status'].describe()

In [ ]:
labels = ('No canceló', 'Canceló')
sns.set(font_scale=1)
ax = sns.countplot(data=hotel, x= "booking_status", palette='deep')
ax.set_xticklabels(labels)
ax.set_xlabel('Estado de la reserva')     #label x
ax.set_ylabel('Numero de reservas')     #label y
ax.set_title('Cantidad de reservas canceladas y no canceladas')  #título
plt.show()

In [ ]:
hotel['booking_status'].value_counts().mul(100)/len(hotel)

Como vemos, nuestra variable target "booking_status" muestra que el 67% de las reservas no fueron canceladas, mientras que el 37% si se cancelaron.

##### Variable "no_of_special_requests"

Tomaremos percentiles pequeños ya que los valores analizados son pequeños.

In [ ]:
hotel['no_of_special_requests'].describe()

In [ ]:
sns.set(font_scale=1)
fig, ax = plt.subplots(figsize=(15, 5))
ax = sns.histplot(hotel.no_of_special_requests, binwidth= 1)
ax.set_xlabel('Pedidos Especiales')   
ax.set_title('Histograma de Pedidos Especiales')      
plt.xticks(np.arange(0, 8, 1), rotation = 45)
plt.show()

##### Variable "type_of_meal_plan"

In [ ]:
hotel.type_of_meal_plan.value_counts()

0 : Sin Comidas.    
1 : Solo Desayuno.  
2 : Media Pensión.  
3 : Pensión Completa

In [ ]:
sns.set(font_scale=1)
ax = sns.countplot(data=hotel, x= "type_of_meal_plan", palette='deep', order=[0,1,2,3]) 
ax.set_xlabel('Régimen de comidas')    #label x
ax.set_ylabel('Número de reservas')     #label y
ax.set_title('Reservas por régimen de comidas')  #título       
plt.show()

In [ ]:
hotel_meal = hotel.groupby(['type_of_meal_plan'])['type_of_meal_plan'].count()

In [ ]:
hotel_meal = hotel.type_of_meal_plan.value_counts().mul(100)/len(hotel) 
hotel_meal

Mediante nuestro gráfico y calculos podemos determinar que el 77% de reservas aproximadamente las realizan con un plan de comidas de solo desayuno. En contra posición, vemos que el regimen completo de comidas solo se solicita con un 0.01% de reservas. Adicionalmente tenemos un 14% para las reservas sin plan de comidas y un 9% de reservas con media pensión.

##### Variable "no_of_adults"

Tomaremos percentiles pequeños ya que los valores analizados son pequeños.

In [ ]:
hotel['no_of_adults'].describe(percentiles=[0.01, 0.1, .25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.99])

In [ ]:
sns.set(font_scale=1)
ax = sns.boxplot(hotel.no_of_adults)
ax.set_xlabel('Adultos') 
ax.set_title('Adultos en las reservas') 
plt.show()

Vemos en la estadistica que el 90% de las reservas son para 2 adultos, lo que nos lleva a ver una caja muy colapsada sobre ese valor.
Los valores menores o mayores a 2 son vistos como outliers.

##### Variable "arrival_month"

In [ ]:
hotel['arrival_month'].describe()

In [ ]:
hotel['arrival_month'].unique()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
sns.set(font_scale=1)
ax = sns.countplot(data=hotel, x= "arrival_month", palette= None, color = '#4c72b0')
ax.set_xlabel('Mes')    
ax.set_ylabel('Reservas')     
ax.set_title('Reservas por mes')       
plt.show()

A partir de la gráfica vemos como desde Enero a Abril las reservas van en aumento, para luego mantener una cierta estacionalidad hasta Agosto, mes donde se disparan las reservas en coincidencia con el verano europeo. Este crecimiento se mantiene hasta Octubre, y a partir de Noviembre las reservas caen practicamente a la mitad motivados por las festividades y la epoca invernal.

#### Analisis Bivariado

##### Tasa de Cancelación

In [ ]:
hotel_pivot = pd.pivot_table(hotel.reset_index(), columns=['arrival_month', 'arrival_year'], 
        index=['booking_status'],
        values='index',
        aggfunc=len, margins=True, margins_name='subtotales')
hotel_pivot

In [ ]:
Tasa_cancelacion = (hotel_pivot.iloc[2] - hotel_pivot.iloc[0])/hotel_pivot.iloc[2]*100

hotel_cancelacion = pd.DataFrame(Tasa_cancelacion)
hotel_cancelacion.head()

In [ ]:
hotel_cancelacion = hotel_cancelacion.reset_index()        #reseteo el indice
hotel_cancelacion.rename({0: 'Porcentaje'}, axis=1, inplace=True)   # renombro columna
hotel_cancel=hotel_cancelacion.drop(hotel_cancelacion.index[-1])          # elimino ultima fila que tiene el subtotal (marginal)
hotel_cancel.tail()

In [ ]:
# Diccionario con los nombres de los meses y su correspondiente formato en numeros
mes = { 'Enero':1,
        'Febrero':2,
        'Marzo':3,
        'Abril':4,
        'Mayo':5,
        'Junio':6,
        'Julio':7,
        'Agosto':8,
        'Septiembre':9,
        'Octubre':10,
        'Noviembre':11,
        'Diciembre':12
        }

In [ ]:
hotel_cancel.replace({"arrival_month": mes}, inplace=True)                #reemplazo el mes en texto por numero (definido antes)
hotel_cancel['Date'] = pd.to_datetime({'year':hotel_cancel['arrival_year'],              #creo columna date y la convierto a datetime
                                    'month':hotel_cancel['arrival_month'],
                                    'day':1
                                    })
hotel_cancel = hotel_cancel.drop(['arrival_month', 'arrival_year'], axis=1)
hotel_cancel.head()

In [ ]:
hotel_cancel.dtypes

In [ ]:
hotel_cancel = hotel_cancel.sort_values(by='Date', ascending=True) 
hotel_cancel.head()

In [ ]:
sns.set(font_scale=1)
fig, ax = plt.subplots(figsize=(15,8))
ax.plot(hotel_cancel['Date'], hotel_cancel['Porcentaje'], marker='o', linestyle='--')
ax.set_title("Tasa de cancelacion por mes")
ax.set_xlabel("Mes")
ax.set_ylabel("Porcentaje (%)")
plt.show()

In [ ]:
max_cancel = hotel_cancel['Porcentaje'].max()
min_cancel = hotel_cancel['Porcentaje'].min()
print('El porcentaje mínimo de cancelación es: ', min_cancel, ' y el máximo es ', max_cancel)

Se calculó la tasa de cancelación de reservas por mes y se graficó para ver variaciones en el tiempo. El porcentaje varió entre el 2% y 67%. El valor más bajo se observa durante la mitad del primer año de registro y luego la tasa de cancelación aumenta oscilando entre el 25 y 50%. Este porcentaje de cancelaciones es considerado muy elevado y termina perjudicando el negocio hotelero.

##### Cancelación y mes de arribo

In [ ]:
fig, cm = plt.subplots(figsize=(15,6)) 
sns.set(font_scale=1)
cm = sns.countplot(x = hotel['arrival_month'], data = hotel, hue='booking_status')
cm.set_xlabel('Mes de arribo')
cm.set_ylabel('Numero de reservas')     
cm.set_title('Reservas canceladas y no canceladas según el mes de arribo')    
plt.show()

##### Cancelación y segmento de mercado

In [ ]:
hotel_aviat = pd.crosstab(hotel.booking_status, hotel.market_Aviation, normalize='columns')
hotel_aviat

In [ ]:
hotel_aviat.plot(kind='bar', stacked=False, color =['#4c72b0', '#dd8452', '#55a868', '#c44e52']) 
ax.set(facecolor = "grey")
plt.grid(color = 'w')
plt.xlabel ('Estado de la reserva')
plt.ylabel ('Frecuencia relativa')
plt.show()

In [ ]:
hotel_compl = pd.crosstab(hotel.booking_status, hotel.market_Complementary, normalize='columns')
hotel_compl

In [ ]:
hotel_compl.plot(kind='bar', stacked=False, color =['#4c72b0', '#dd8452', '#55a868', '#c44e52']) 
ax.set(facecolor = "grey")
plt.grid(color = 'w')
plt.xlabel ('Estado de la reserva')
plt.ylabel ('Frecuencia relativa')
plt.show()

In [ ]:
hotel_corpo = pd.crosstab(hotel.booking_status, hotel.market_Corporate, normalize='columns')
hotel_corpo

In [ ]:
hotel_corpo.plot(kind='bar', stacked=False, color =['#4c72b0', '#dd8452', '#55a868', '#c44e52']) 
ax.set(facecolor = "grey")
plt.grid(color = 'w')
plt.xlabel ('Estado de la reserva')
plt.ylabel ('Frecuencia relativa')
plt.show()

In [ ]:
hotel_offline = pd.crosstab(hotel.booking_status, hotel.market_Offline, normalize='columns')
hotel_offline

In [ ]:
hotel_offline.plot(kind='bar', stacked=False, color =['#4c72b0', '#dd8452', '#55a868', '#c44e52']) 
ax.set(facecolor = "grey")
plt.grid(color = 'w')
plt.xlabel ('Estado de la reserva')
plt.ylabel ('Frecuencia relativa')
plt.show()

In [ ]:
hotel_online = pd.crosstab(hotel.booking_status, hotel.market_Online, normalize='columns')
hotel_online

In [ ]:
hotel_online.plot(kind='bar', stacked=False, color =['#4c72b0', '#dd8452', '#55a868', '#c44e52']) 
ax.set(facecolor = "grey")
plt.grid(color = 'w')
plt.xlabel ('Estado de la reserva')
plt.ylabel ('Frecuencia relativa')
plt.show()

##### Cancelación y lead time

In [ ]:
cl = sns.boxplot(x = "booking_status", y ="lead_time", data = hotel) 
sns.set(font_scale=1)
cl.set_xlabel('Estado de la reserva')
cl.set_xticklabels(labels)
cl.set_ylabel('Dias en lista de espera')    
cl.set_title('Dias en lista de espera vs Estado de la Reservas')      
plt.show()

In [ ]:
labels=('No canceló', 'Canceló')
sns.set(font_scale=1)
ax = sns.barplot(x='booking_status', y='lead_time', data=hotel) 
ax.set_xticklabels(labels)
ax.set_title('Dias en lista de espera vs Estado de la Reservas')
ax.set_xlabel('Estado de la reserva')
ax.set_ylabel('Dias en lista de espera')
plt.show()

Se observa que las personas que cancelaron la reserva tuvieron más tiempo de espera para reservar que los clientes que no cancelaron.

##### Cancelacion y requisito de estacionamiento

In [ ]:
hotel.required_car_parking_space.value_counts()

In [ ]:
labels=('No canceló', 'Canceló')
sns.set(font_scale=1)
ax = sns.barplot(x='booking_status', y='required_car_parking_space', data=hotel) 
ax.set_xticklabels(labels)
ax.set_title('Requisito de estacionamiento vs Estado de la Reservas')
ax.set_xlabel('Estado de la reserva')
ax.set_ylabel('Solicitud de estacionamiento')
plt.show()

Vemos que la gran mayoria de las personas que solicitaron lugar de estacionamiento con su reserva no cancelaron la misma.

##### Cancelación y total de noches

In [ ]:
labels=('No canceló', 'Canceló')
sns.set(font_scale=1)
ax = sns.barplot(x='booking_status', y='total_nights', data=hotel, estimator = median)
ax.set_xticklabels(labels)
ax.set_title('Longitud de la estadía (mediana) según estado de la reserva')
ax.set_xlabel('Estado de la reserva')
ax.set_ylabel('Longitud de la estadía')
plt.show()

In [ ]:
labels=('No canceló', 'Canceló')
sns.set(font_scale=1)
ax = sns.barplot(x='booking_status', y='total_nights', data=hotel)
ax.set_xticklabels(labels)
ax.set_title('Longitud de la estadía (promedio) según estado de la reserva')
ax.set_xlabel('Estado de la reserva')
ax.set_ylabel('Longitud de la estadía')
plt.show()

La mediana y el promedio de la longitud de la estadía en días de semana no se verian relacionadas con la probabilidad de cancelar ya que son similares en ambos grupos. 

#### Análisis Multivariado

In [ ]:
hotel.info()

##### *Correlación entre variables cuantitativas según variable target "booking_status"*

In [ ]:
hotel_cuant= hotel.select_dtypes(include=np.number)
hotel_cuant.info()

In [ ]:
status=hotel['booking_status']
status

In [ ]:
hotel_cuant['status']=status
hotel_cuant.info()

In [ ]:
sns.pairplot(hotel_cuant, hue='status')

- La variable "previous_booking" presenta valores altos para el grupo que canceló.
- En la variable "total_nights" se visualizan 2 nubes bien diferenciadas entre Canceló y no Cancelo.
- La variable "arrival_date" tiene un comportamiento similar a "total_nights".
- La variable "lead_time" se distribuye de forma diferente en los grupos Canceló/No canceló.
- En las variables restantes es dificil visualizar una diferenciación entre los usuarios que cancelaron y los que no. Visualizare "no_of_special_requests"

##### "previous_booking" y "room_type_reserved" vs "booking_status"

In [ ]:
plt.figure(figsize=(10,10))
labels=('No canceló', 'Canceló')
ax = sns.boxplot(x = "booking_status", y ='previous_booking', hue = 'repeated_guest', data = hotel)
ax.set_xlabel('Estado de la reserva')
ax.set_ylabel('Lead time') 
ax.set_xticklabels(labels)   
ax.set_title('Reservas previas por tipo de habitación en Reservas canceladas y no canceladas')  
plt.show()

In [ ]:
labels=('No canceló', 'Canceló')
ax = sns.barplot(x = "booking_status", y ='previous_booking', hue = 'repeated_guest', data = hotel, estimator = median)
ax.set_xticklabels(labels)
ax.set_title('Reserva previa (mediana) por tipo de habitación y estado de la reserva')
ax.set_xlabel('Estado de la reserva')
plt.show()

### Correlación de las variables

In [ ]:
correlation = hotel.corr().round(2)
plt.figure(figsize = (14,7))
sns.heatmap(correlation, annot = True, cmap = 'YlOrBr')

No se visualiza una correlación muy fuerte entre las variables, y las que se visualizan es porque estan derivadas de otras variable (por ejemplo total_nights con no_of_weekend_nights). Se mantienen momentaneamente para realizar el análisis.

## Storytelling

La industria hotelera es una de las más afectadas en cuanto a cancelación de reservas se refiere. Los hoteles pueden disminuir significativamente sus ingresos cuando los viajeros cancelan las resevas en situaciones que no les permiten completar las habitaciones nuevas con nuevos huespedes. Al privarse de vender estas habitaciones, como se mencionaba anteriormente , la rentabilidad de los hoteles disminuye ya que se generan costos adicionales para huespedes que no cumpliran con la reserva.
Poder establecer un modelo que ayude a predecir las reservas que se cancelarán es una condicion sine qua non para mejorar la rentabilidad y eficiencia del negocio.

El set de datos incluye cerca de 36000 reservas entre canceladas y no canceladas junto con variables referidas a los huespedes, a las características que ofrece el hotel, e información específica de las reservas. Este dataset nos pormitira desarrollar a futuro un modelo de predicción que nos permita asistir a la industria hotelera.

Se identificaron algunas relaciones entre el estado de la reserva y ciertas variables:

* A mayor dias de espera para la llegada al hotel, mayor tasa de cancelaciones.
* El segmento de mercado "Complementary" no registra cancelaciones
* Las solicitudes de espacio de estacionamiento no registran cancelaciones de reservas
* Julio es el mes donde se registran mayor cantidad de cancelaciones en relación a la cantidad de reservas realizadas
* A mayores requerimientos especiales menor es la tasa de cancelaciones
* El indice de huespedes recurrentes es muy bajo.
* Se suma el total de adultos y niños.
* Se suman las noches de la semana y el fin de semana
* El regimen de comidas 2 registra una tasa de cancelación similar a las no canceladas.
* Las reservas de habitación tipo 1 son la de mayor cantidad de reservas.

Todas las variables se incluiran en el dataset para el armado del futuro modelo de clasificación a excepción de:

* Date por ser de tipo datetime.

## Preparación del modelo

### Librerías necesarias

In [ ]:
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc   


In [ ]:
hotel.info()

Se descartan las variables que no se utilizarán en el modelo.

- 'total_people' porque quiero precisar la cantidad de adultos y niños. 
- 'arrival_year' porque es poco tiempo de análisis.
- 'arrival_date' tiene muchas clases.
- 'avg_price_per_room' tiene demasiadas clases.
- 'Date' es de tipo fecha.

In [ ]:
hotel_clasif = hotel[['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
                       'room_type_reserved', 'lead_time', 'arrival_month', 'repeated_guest', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                         'no_of_special_requests', 'total_nights', 'previous_booking', 'market_Aviation', 'market_Complementary',
                           'market_Corporate', 'market_Offline', 'market_Online', 'booking_status' ]]
hotel_clasif.info()

Concateno 'market_segment_type' porque me sera mas útil de esta forma que separada por market.

In [ ]:
hotel_clasif = pd.concat([hotel_clasif,hotel_orig['market_segment_type']], axis= 1)

In [ ]:
hotel_clasif = hotel_clasif.drop(['market_Aviation', 'market_Complementary', 'market_Corporate', 'market_Offline', 'market_Online'], axis= 1)

In [ ]:
hotel_clasif.type_of_meal_plan = pd.to_numeric(hotel_clasif.type_of_meal_plan)
hotel_clasif.required_car_parking_space = pd.to_numeric(hotel_clasif.required_car_parking_space)
hotel_clasif.repeated_guest = pd.to_numeric(hotel_clasif.repeated_guest)
hotel_clasif.booking_status = pd.to_numeric(hotel_clasif.booking_status)

hotel_clasif.info()

### Encoding y normalización de columnas numéricas

In [ ]:
colunumeros = hotel_clasif.select_dtypes(include=['float64', 'int']).columns.to_list()
colucateg = hotel_clasif.select_dtypes(include=['object']).columns.to_list()

In [ ]:
colunumeros

In [ ]:
preprocessor = ColumnTransformer(
                   [('scale', RobustScaler(), colunumeros),                                      
                    ('onehot', OneHotEncoder(handle_unknown='ignore'), colucateg)],
                remainder='passthrough')

In [ ]:
hotel_clasif_prepro = preprocessor.fit_transform(hotel_clasif)

Como 'ColumnTransformer' me devuelve un numpy array, convierto el preprocesado en un dataframe para poder recuperar los nombres de las variables. Para ello, utilizo 'OneHotEncoder' para ordenarlos alfabeticamente.

In [ ]:
encoding_cat = preprocessor.named_transformers_['onehot'].get_feature_names_out(colucateg)
labels = np.concatenate([colunumeros, encoding_cat])
hotel_clasif_prepro = pd.DataFrame(hotel_clasif_prepro, columns=labels)
hotel_clasif_prepro.info()

### Dataset Final

Mi dataset final para realizar los modelos de clasificación será hotel_clasif_prepro. Verifico que las variables obtenidas para trabajar son todas numéricas y no tengo datos faltantes.

In [ ]:
hotel_clasif_prepro.head()

In [ ]:
hotel_clasif_prepro.shape

In [ ]:
hotel_clasif_prepro.describe()

In [ ]:
hotel_clasif_prepro.isna().any().any()

## Algoritmo de Clasificación - Entrenamiento y Evaluación

### Sets de train (entrenamiento) y test (evaluación)

Separo las variables de la variable target

In [ ]:
X = hotel_clasif_prepro.drop("booking_status", axis=1)
y = hotel_clasif_prepro.booking_status

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) #random_state es la semilla.

Se separo el dataset un 70% para entrenamiento y un 30% para testeo.

### KNN, Regresión logística y Random Forest.

Evaluaremos y compararemos los 3 modelos de clasifición mencionados. Realizaremos tambien una cross-validation (validación cruzada) para ajustar los modelos y calcular la presición de cada uno (accuracy).

In [ ]:
pipeline_lr = Pipeline([('LR',  LogisticRegression(max_iter=5000, random_state=42))])
pipeline_fr = Pipeline([ ('RF', RandomForestClassifier(random_state=42))])
pipeline_knn = Pipeline([ ('KNN', KNeighborsClassifier()) ])

In [ ]:
pipelines = [pipeline_lr, pipeline_fr, pipeline_knn]
pipelines

In [ ]:
pip_dict = {0: 'Regresion Logistica', 1: 'Random Forest', 2: 'KNN' }

In [ ]:
def val_cross(X_train,X_test, y_train, y_test):    

    modeloutcomes = []
    modelnames = []
    for i,model in enumerate(pipelines):            
        v_results = cross_val_score(model, X_train, y_train, cv = 3, 
                                     scoring='accuracy', n_jobs = -1, verbose = 0)
        print(pip_dict[i], v_results.mean())
        modeloutcomes.append(v_results)
        modelnames.append(pip_dict[i])
        
    print(modeloutcomes) 
    
    fig = plt.figure() 
    ax = fig.add_subplot(111)
    plt.boxplot(modeloutcomes)
    ax.set_xticklabels(modelnames)
    

val_cross(X_train,X_test, y_train, y_test)

De acuerdo a la grafica y los valores obtenidos podemos verificar que con Random Forest obtenemos una mayor presición que los otros 2 modelos.
De igual forma, calcularemos las metricas de *Presicion*, *Recall*, *F1 Score* y *Curva ROC* para poder obtener una mejor visión sobre la calidad de los diferentes modelos.

Como necesitamos que nuestro modelo prediga la mayor cantidad posible de positivos (reservas a cancelar), nos enfocaremos en la métrica de Precision

### Métricas

Creo un dataframe con los nombres de las columnas para almacenar los resultados de las métricas.

In [ ]:
metricas = pd.DataFrame(columns = ['Model','Accuracy','Precision','Recall','F1_Score', 'AUC ROC'])    
metricas = metricas[['Model','Accuracy','Precision','Recall','F1_Score', 'AUC ROC']]

In [ ]:
for i,model in enumerate(pipelines):     #recorre la lista de pipelines
    try:
        trainedmodel = model.fit(X_train,y_train)        #entrena el modelo
    
        # Realizamos las predicciones de cada modelo
        ypredict = trainedmodel.predict(X_test)
    
        # Calculamos las métricas
        prec = precision_score(y_test,ypredict)
        accur = accuracy_score(y_test, ypredict)
        rec = recall_score(y_test, ypredict)
        f1score = f1_score(y_test,ypredict)
        classreport = classification_report(y_test,ypredict)
        confMat = confusion_matrix(y_test,ypredict)

        # Imprimimos resultados
        print('\n******  '+pip_dict[i]+'  *****')
        print('La precisión es: {}'.format(prec))
        print('\nEl Informe de Clasificación es:\n {}'.format(classreport))
        print('La Matriz de Confusión es:\n ')
        
        #Ploteamos la Matriz de confusión
        disp = ConfusionMatrixDisplay(confusion_matrix=confMat)
        disp.plot()
        plt.title('Matriz de Confusión')
        plt.show()
        

        class_probabilities = trainedmodel.predict_proba(X_test)
        preds = class_probabilities[:, 1]

        fpr, tpr, threshold = roc_curve(y_test, preds)
        roc_auc = auc(fpr, tpr)

        # AUC (área bajo la curva ROC)
        print(f"\n Área bajo curva ROC del modelo (AUC ROC): {roc_auc}")

        # Gráfica de la Curva ROC
        plt.title('Receiver Operating Characteristic')
        plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()

        #Guardamos los resultados en el nuevo dataframe para luego visualizar
        metricas = metricas.append({'Model':pip_dict[i], 'Accuracy':accur, 'Precision':prec, 
                                    'Recall':rec, 'F1_Score':f1score, 'AUC ROC':roc_auc}, ignore_index=True)

    except Exception as e:
        print("Ocurrió un error al ejecutar el modelo:", pip_dict[i])
        print("Error:", e)

In [ ]:
metricas.head()

*Vemos, de acuerdo a la tabla, que nuestro mejor modelo para poder realizar una predicción es el Random Forest*.